<a href="https://colab.research.google.com/github/lior9005/deep-learning/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADL final project


---


**Authors:**
> Lior Sharony 316380138


> Noam Zigler ********


> Eden Miran *********


In [ ]:
# imports


# Eval mode
eval_mode = True

# Data & Preprocessing

In [ ]:
#datasets, dataloaders, transformations


#subset of CIFAR10/FashionMNIST


# Models

In [ ]:
# Baseline model class


# OSR model class



# Training

In [ ]:
if not (eval_mode)

  # Training procedure


  # Training and validation loss and accuracy for MNIST plot



# Evaluation - CIFAR10/Fashion-MNIST

In [ ]:
if (eval_mode)
  # Load the trained models weights


# Baseline results

**OSR approach:**

---



In [ ]:
# OOD results


# OSR results


# t-SNE and/or PCA visualization (optional)


# Additional figures (optional)

# Evaluation - Unknown Dataset

In [ ]:
# placeholder for the combined OSR dataset and dataloader


# Load the trained models weights


# OOD results


# OSR results